In [19]:
%load_ext autoreload
%autoreload 2
import sys, os
from itertools import product
import scipy.sparse
from scipy.sparse.linalg import dsolve
from scipy.sparse import csr_matrix
from collections import ChainMap
sys.path.append(os.path.dirname(sys.path[0]))
import torch
import numpy as np
from neural_control.controllers import DualFullyConnectedRegressionController
import pickle
np.random.seed(0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# Commented out: first set of configurations (high service level)
ce_list = [5, 10, 20]
lr_list = [2, 3] #[2, 3, 4]
b_list = [85] #[495, 95]
h = 15 #5
d_max_list = [4, 8]

### Breadth First Search on DP States
We need to ensure that we calculate the RMSE on states that the DP actually visits (transient states should be left out).
For this, we implement a Breadth First Search algorithm.

First, we define a function that returns the states in which we may transit from a given state $s$.

In [3]:
def neighboring_states(state):
    neighbors = []
    qr, qe = qf_dp[state]
    ip_e = state[0] + qe + state[1]
    pipeline = state[2:] if lr > 2 else qr
    for demand in range(d_max+1):
        ipe_new = ip_e - demand
        this_state = (ipe_new, *pipeline, qr) if lr > 2 else (ipe_new, qr)
        neighbors.append(this_state)
    return neighbors

In [4]:
def bfs(node=(0, )*2): #function for BFS
    visited, queue = [], []
    visited.append(node)
    queue.append(node)

    while queue:          # Creating loop to visit each node
        m = queue.pop(0) 
#         print (m) 

        for neighbor in neighboring_states(m):
            if neighbor not in visited:
                visited.append(neighbor)
                queue.append(neighbor)
                
    return visited
                
# qf_dp2 = {x:y for x, y in qf_dp.items() if x in visited}

### IDINN RMSE Calculation
We calculate
$$ {\rm{RMSE}}_m = \sqrt{\frac{1}{\lvert \mathcal{S}_{\rm{DP}} \rvert} \sum_{s \in\mathcal{S}_{\rm{DP}}} \left(q_{\rm {DP}}^{\rm{r}}(\mathbf{s})-q_m^{\rm{r}}(\mathbf{s})\right)^2 + \left(q_{\rm{DP}}^{\rm{e}}(\mathbf{s})-q_m^{\rm{e}}(\mathbf{s})\right)^2}\,,$$

for all the states of DP that are common with IDINN.

In [7]:
for lr in lr_list:
    for ce in ce_list:
        for b in b_list:
            for d_max in d_max_list:
                instance = f'lr={lr}ce={ce}b={b}h={h}u={d_max}'
                f_name = f'dp_state_output_' + instance + '.p'
#                 f_name = f'dp_state_output_lr={lr}ce={ce}b={b}h={h}u={d_max}.p'
                try:
                    f_name = os.path.join('../', 'sourcing_models', 'dp_state_output', f_name)
                    qf_dp = pickle.load(open(f_name, 'rb'))
                    
                except(OSError, IOError) as e:
                    print(f'file {f_name} does not exist.')
                    break
                visited = bfs((0, )*lr)
                qf_dp2 = {x:y for x, y in qf_dp.items() if x in visited}
                
#                 f_name_nn = f'nnc_state_output_lr={lr}ce={ce}b={b}h={h}u={d_max}.p'
                f_name_nn = f'nnc_state_output_' + instance + '.p'
                f_name_nn = os.path.join('../', 'sourcing_models', 'nn_state_output', f_name_nn)
                qf_nn = pickle.load(open(f_name_nn, 'rb'))
                qf_nn2 = {(x[0],)+x[1]: y for x, y in qf_nn.items() if (x[0],)+x[1] in qf_dp2}
                
                qf_diff = np.array([sum( (qf_dp2[x][y]-qf_nn2[x][y])**2 for y in range(2)) for x in qf_dp2])
                rmse = np.sqrt(qf_diff.mean())
                print(f'instance: {instance} RMSE: {round(rmse, 2)}')


instance: lr=2ce=5b=85h=15u=4 RMSE: 0.32
instance: lr=2ce=5b=85h=15u=8 RMSE: 0.41
instance: lr=2ce=10b=85h=15u=4 RMSE: 0.25
instance: lr=2ce=10b=85h=15u=8 RMSE: 0.53
instance: lr=2ce=20b=85h=15u=4 RMSE: 0.4
instance: lr=2ce=20b=85h=15u=8 RMSE: 0.46
instance: lr=3ce=5b=85h=15u=4 RMSE: 0.6
instance: lr=3ce=5b=85h=15u=8 RMSE: 1.27
instance: lr=3ce=10b=85h=15u=4 RMSE: 0.36
instance: lr=3ce=10b=85h=15u=8 RMSE: 0.57
instance: lr=3ce=20b=85h=15u=4 RMSE: 0.45
instance: lr=3ce=20b=85h=15u=8 RMSE: 0.55


### Neural Network Calculations
Preliminary. These are necessary to extract the states and save the files. Not necessary to run this part unless new instances are added.

In [23]:
nnc_hyperparameters = dict(
    n_hidden_units = [128,64,32,16,8,4]
)
nnc_hyperparameters['n_activations'] = [torch.nn.CELU(alpha=1)]*(2 + len(nnc_hyperparameters['n_hidden_units']))

In [24]:
def make_fcc(nnc_hyperparameters):
    fcc = DualFullyConnectedRegressionController(lr=lr, le=0, 
                                         n_hidden_units= nnc_hyperparameters ['n_hidden_units'],
                                         activations=nnc_hyperparameters['n_activations'])
    return fcc

def load_model(f_name):
    best_model_load = torch.load(f_name, map_location='cpu')
    fcc = make_fcc(nnc_hyperparameters)
    fcc.load_state_dict(best_model_load)
    return fcc

In [33]:
def make_states(lr, d_max):
    dim_pipeline = lr
    min_ip = int(d_max * lr)
    max_ip = int((lr + 1) * (d_max + 1) + d_max)
    states = list(product(range(-min_ip, max_ip + 1), *(range(int(d_max) + 1),) * int(dim_pipeline)))
    return states

In [117]:
rem = len(ce_list) * len(lr_list) * len(b_list) * len(d_max_list)
idx = 0
for ce in ce_list:
    for lr in lr_list:
        for b in b_list:
            for d_max in d_max_list:
                idx = idx + 1
                f_name = f'nnc_best_model_direct_lr={lr}_ce={ce}_b={b}_h=5_u0{d_max}.pt'
                print(f'Loading file {f_name}. Still {rem - idx} to go.')
                f_name = os.path.join('../', 'sourcing_models', 'trained_neural_nets', f_name)
                fcc = load_model(f_name)
                states = make_states(lr, d_max)
                qf_nnc = {}
                state_counter = {}
                for state in states:
                    inv = float(state[0])
                    pipeline = [float(el) for el in state[1:]]
                    qr, qe = fcc(torch.tensor(10.0), torch.tensor([[inv]]), torch.tensor([pipeline]), torch.tensor([[10.0]]))
                    val = (qr.detach().item(), qe.detach().item())

                    compressed_state = (state[0] + state[1], state[2:])
                    if compressed_state not in qf_nnc:
                        qf_nnc[compressed_state] = val
                        state_counter[compressed_state] = 1
                    else:
                        # If there are different orders for each compressed state, we take the average
                        state_counter[compressed_state] = state_counter[compressed_state] + 1
                        delta = 1/state_counter[compressed_state]
                        q_r_new = qf_nnc[compressed_state][0] + delta * (val[0] - qf_nnc[compressed_state][0])
                        q_e_new = qf_nnc[compressed_state][1] + delta * (val[1] - qf_nnc[compressed_state][1])
                        qf_nnc[compressed_state] = (q_r_new, q_e_new)
                out_f_name = f'nnc_state_output_lr={lr}ce={ce}b={b}h={h}u={d_max}.p'
                out_f_name = os.path.join('../', 'sourcing_models', 'nn_state_output', out_f_name)
                pickle.dump(qf_nnc, open(out_f_name, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
                

Loading file nnc_best_model_direct_lr=2_ce=5_b=495_h=5_u04.pt. Still 35 to go.
Loading file nnc_best_model_direct_lr=2_ce=5_b=495_h=5_u08.pt. Still 34 to go.
Loading file nnc_best_model_direct_lr=2_ce=5_b=95_h=5_u04.pt. Still 33 to go.
Loading file nnc_best_model_direct_lr=2_ce=5_b=95_h=5_u08.pt. Still 32 to go.
Loading file nnc_best_model_direct_lr=3_ce=5_b=495_h=5_u04.pt. Still 31 to go.
Loading file nnc_best_model_direct_lr=3_ce=5_b=495_h=5_u08.pt. Still 30 to go.
Loading file nnc_best_model_direct_lr=3_ce=5_b=95_h=5_u04.pt. Still 29 to go.
Loading file nnc_best_model_direct_lr=3_ce=5_b=95_h=5_u08.pt. Still 28 to go.
Loading file nnc_best_model_direct_lr=4_ce=5_b=495_h=5_u04.pt. Still 27 to go.
Loading file nnc_best_model_direct_lr=4_ce=5_b=495_h=5_u08.pt. Still 26 to go.
Loading file nnc_best_model_direct_lr=4_ce=5_b=95_h=5_u04.pt. Still 25 to go.
Loading file nnc_best_model_direct_lr=4_ce=5_b=95_h=5_u08.pt. Still 24 to go.
Loading file nnc_best_model_direct_lr=2_ce=10_b=495_h=5_u0

### Dual Sourcing Calculations

In [9]:
from sourcing_models.lib import dual_sourcing_state_output

This line is for a trial/debugging, it is not really necessary.

In [7]:
ce, lr, b, h, d_max = ce_list[0], lr_list[0], b_list[0], 5, d_max_list[1]

In [10]:
def find_gamma_sigma():
    """Returns the parameters Gamma and Sigma Sun & Van Mieghem (2019), MSOM, eq. (42)
    assuming U{a, b} demand"""
    sigma = np.sqrt(((d_max+1)**2-1)/12)
    gamma = d_max / (2*sigma)
    return gamma, sigma

In [11]:
def find_cdi_param_arrays(gamma, sigma, bandwidth=10):
    mu = d_max / 2
    cost_ratio = (b - h) / (b + h)
    u1 = mu + cost_ratio * gamma * sigma
    u2 = (lr+1)*mu + cost_ratio*(lr+1)*gamma*sigma
    u3 = mu + cost_ratio*gamma*sigma
    u1_arr = range(int(max(u1-bandwidth, 0)), int(u1+bandwidth))
    u2_arr = range(int(max(u2-bandwidth, 0)), int(u2+bandwidth))
    u3_arr = range(int(max(u3-bandwidth, 0)), int(u3+bandwidth))
    return u1_arr, u2_arr, u3_arr

In [20]:
# dual_sourcing_state_output.capped_dual_index_parameters(u1_arr, u2_arr, u3_arr, ce=ce, lr=lr, h=h, b=b, T=2000)
cdi_optimal = {}
cnt = 0
T = 2000
for ce in ce_list:
    for lr in lr_list:
        for b in b_list:
            for d_max in d_max_list:
                instance = f'lr={lr}ce={ce}b={b}h={h}u={d_max}'
                gamma, sigma = find_gamma_sigma()
                u1_arr, u2_arr, u3_arr = find_cdi_param_arrays(gamma, sigma)
                demand_distr = np.random.randint(0, high=d_max+1, size=T)
                u1, u2, u3, cost = dual_sourcing_state_output.capped_dual_index_parameters(u1_arr, u2_arr, u3_arr, 
                                                           ce=ce, lr=lr, h=h, b=b, demand_distribution=demand_distr, T=T)
                cdi_optimal[instance] = (u1, u2, u3, cost)
                cnt = cnt + 1
                print(f'Completed {cnt} out of 36 instances. Instance: {instance} Cost: {cost} Solution: {(u1, u2, u3)}')
                

Completed 1 out of 36 instances. Instance: lr=2ce=5b=85h=15u=4 Cost: 39.435 Solution: (4, 4, 1)
Completed 2 out of 36 instances. Instance: lr=2ce=5b=85h=15u=8 Cost: 71.69 Solution: (7, 11, 2)
Completed 3 out of 36 instances. Instance: lr=3ce=5b=85h=15u=4 Cost: 39.72 Solution: (4, 4, 1)
Completed 4 out of 36 instances. Instance: lr=3ce=5b=85h=15u=8 Cost: 72.085 Solution: (7, 17, 2)
Completed 5 out of 36 instances. Instance: lr=2ce=10b=85h=15u=4 Cost: 43.5975 Solution: (3, 6, 2)
Completed 6 out of 36 instances. Instance: lr=2ce=10b=85h=15u=8 Cost: 80.7075 Solution: (6, 13, 3)
Completed 7 out of 36 instances. Instance: lr=3ce=10b=85h=15u=4 Cost: 45.3975 Solution: (3, 8, 1)
Completed 8 out of 36 instances. Instance: lr=3ce=10b=85h=15u=8 Cost: 80.225 Solution: (6, 17, 3)
Completed 9 out of 36 instances. Instance: lr=2ce=20b=85h=15u=4 Cost: 50.7525 Solution: (3, 8, 2)
Completed 10 out of 36 instances. Instance: lr=2ce=20b=85h=15u=8 Cost: 90.78 Solution: (6, 15, 4)
Completed 11 out of 36 inst

In [21]:
# Check again: state in CDI, NN, DP is like the one below.
def inv_position(state, k):
    """
    state: (I0+qr_{lr}, qr_{lr-1}, ..., qr_{t-1})
    return: Current inventory + all orders that will arrive by period k
    """
    return sum(state[:k+1])

def get_cdi_action(state, u1, u2, u3):
    ItLm1 = inv_position(state, lr)
    Itt = inv_position(state, 0)
    qslow = min(u3, max(u2-ItLm1, 0))
    qfast = max(u1-Itt, 0)
    return qslow, qfast

def uncover_cdi_states(qf_cdi):

    all_states = [qf_cdi]
    search = True
    while search:
        ret = dict(ChainMap(*all_states))
        qf_new = {}
        for state in all_states[-1]:
            qr, qe = ret[state]
            ip_e = state[0] + qe + state[1]
            pipeline = state[2:] if lr > 2 else qr
            for demand in range(d_max+1):
                ipe_new = ip_e - demand
                new_state = (ipe_new, *pipeline, qr) if lr > 2 else (ipe_new, qr)
                if new_state not in ret:
                    # qf_new[new_state] = get_cdi_action(new_state, *cdi_exact_optimal[instance][:3])
                    qf_new[new_state] = get_cdi_action(new_state, *cdi_optimal[instance][:3])
        if qf_new:
            all_states.append(qf_new.copy())
        else:
            search = False
    return ret

In [22]:
for lr in lr_list:
    for ce in ce_list:
        for b in b_list:
            for d_max in d_max_list:
                instance = f'lr={lr}ce={ce}b={b}h={h}u={d_max}'
                f_name = f'dp_state_output_' + instance + '.p'
                try:
                    f_name = os.path.join('../', 'sourcing_models', 'dp_state_output', f_name)
                    qf_dp = pickle.load(open(f_name, 'rb'))
                    
                except(OSError, IOError) as e:
                    print(f'file {f_name} does not exist.')
                    break
                visited = bfs((0, )*lr)
                qf_dp2 = {x:y for x, y in qf_dp.items() if x in visited}
                
                qf_cdi = {}                
                u1, u2, u3, cost = cdi_optimal[instance] #cdi_exact_optimal[instance]
                for state in qf_dp2:
                    qf_cdi[state] = get_cdi_action(state, u1, u2, u3)
                
                qf_cdi = uncover_cdi_states(qf_cdi)

                qf_diff_cdi = np.array([sum( (qf_dp2[x][y]-qf_cdi[x][y])**2 for y in range(2)) for x in qf_dp2])
                rmse = np.sqrt(qf_diff_cdi.mean())
                print(f'instance: {instance} CDI RMSE: {round(rmse, 2)}')


instance: lr=2ce=5b=85h=15u=4 CDI RMSE: 0.6
instance: lr=2ce=5b=85h=15u=8 CDI RMSE: 0.31
instance: lr=2ce=10b=85h=15u=4 CDI RMSE: 0.69
instance: lr=2ce=10b=85h=15u=8 CDI RMSE: 0.65
instance: lr=2ce=20b=85h=15u=4 CDI RMSE: 0.24
instance: lr=2ce=20b=85h=15u=8 CDI RMSE: 0.79
instance: lr=3ce=5b=85h=15u=4 CDI RMSE: 0.62
instance: lr=3ce=5b=85h=15u=8 CDI RMSE: 0.74
instance: lr=3ce=10b=85h=15u=4 CDI RMSE: 0.69
instance: lr=3ce=10b=85h=15u=8 CDI RMSE: 0.94
instance: lr=3ce=20b=85h=15u=4 CDI RMSE: 0.46
instance: lr=3ce=20b=85h=15u=8 CDI RMSE: 0.96


### Exact Cost calculation

In [13]:
def t(s_s, s_f):
    """Transi tion cost function from state s_s to s_f"""
    inv_pos = s_f[0] - s_s[1]
    cost = h * inv_pos if inv_pos > 0 else - b * inv_pos
    return cost

In [14]:
# Example
ce, lr, b, h, d_max = ce_list[2], lr_list[0], b_list[1], 5, d_max_list[0]
instance = f'lr={lr}ce={ce}b={b}h={h}u={d_max}'
f_name = f'dp_state_output_' + instance + '.p'
#                 f_name = f'dp_state_output_lr={lr}ce={ce}b={b}h={h}u={d_max}.p'
try:
    f_name = os.path.join('../', 'sourcing_models', 'dp_state_output', f_name)
    qf_dp = pickle.load(open(f_name, 'rb'))
    if ((0,)*lr) in qf_dp:
        key = (0,)*lr
    else:
        key = int(len(qf_dp)/2)
    visited = bfs(key)
    qf_dp2 = {x:y for x, y in qf_dp.items() if x in visited}
except(OSError, IOError) as e:
    print(f'file {f_name} does not exist.')

In [18]:
def exact_cost(qf_dp2):
    
    n_states = len(qf_dp2)
    indexes = dict(zip(qf_dp2.keys(), range(n_states)))
    rhs = np.zeros(n_states)
    rhs[-1] = 1
    transition_matrix = np.zeros((n_states, n_states))
    transition_cost = np.zeros_like(transition_matrix)
    
    # Careful! This works for uniform demand only!
    # This is the transition probability from a state
    # s to a reachable state s'.
    prob = 1. / (d_max + 1.)

    for idx, state in enumerate(qf_dp2):
        qr, qe = qf_dp2[state]
    #     partial state update
        ip_e = state[0] + qe + state[1]
        pipeline = state[2:] if lr > 2 else qr
        for demand in range(d_max+1):
            ipe_new = ip_e - demand
            new_state = (ipe_new, *pipeline, qr) if lr > 2 else (ipe_new, qr)
            if new_state in qf_dp2:
                cost = t(state, new_state) + qe * ce
                transition_matrix[idx, indexes[new_state]] = prob
                transition_cost[idx, indexes[new_state]] = cost
            else:
                print(f'Problem from state {state} to state {new_state}.')

    mtx = np.eye(n_states)-transition_matrix
    mtx = mtx[:, 1:]
    mtx = np.hstack((mtx, np.ones((n_states, 1)))).T
    mtx = csr_matrix(mtx)
    ss_probs = dsolve.spsolve(mtx, rhs)
    cost = np.dot(ss_probs, transition_matrix * transition_cost).sum()   
    return cost
#     print(f'instance: {instance} cost: {round(cost, 2)}')

#### Exact cost calculation DP

In [19]:
for lr in lr_list:
    for ce in ce_list:
        for b in reversed(b_list):
            for d_max in d_max_list:
                instance = f'lr={lr}ce={ce}b={b}h={h}u={d_max}'
                f_name = f'dp_state_output_' + instance + '.p'
                try:
                    f_name = os.path.join('../', 'sourcing_models', 'dp_state_output', f_name)
                    qf_dp = pickle.load(open(f_name, 'rb'))
                    if ((0,)*lr) in qf_dp:
                        key = (0,)*lr
                    else:
                        key = int(len(qf_dp)/2)
                    visited = bfs(key)
                    qf_dp2 = {x:y for x, y in qf_dp.items() if x in visited}
                    cost = exact_cost(qf_dp2)
                    print(f'instance: {instance} cost: {round(cost, 2)}')
                except(OSError, IOError) as e:
                    print(f'file {f_name} does not exist.')

instance: lr=2ce=5b=95h=5u=4 cost: 16.77
instance: lr=2ce=5b=95h=5u=8 cost: 32.27
instance: lr=2ce=5b=495h=5u=4 cost: 16.77
instance: lr=2ce=5b=495h=5u=8 cost: 32.27
instance: lr=2ce=10b=95h=5u=4 cost: 19.73
instance: lr=2ce=10b=95h=5u=8 cost: 37.23
instance: lr=2ce=10b=495h=5u=4 cost: 19.73
instance: lr=2ce=10b=495h=5u=8 cost: 37.83
instance: lr=2ce=20b=95h=5u=4 cost: 22.82
instance: lr=2ce=20b=95h=5u=8 cost: 41.63
instance: lr=2ce=20b=495h=5u=4 cost: 23.07
instance: lr=2ce=20b=495h=5u=8 cost: 43.76
instance: lr=3ce=5b=95h=5u=4 cost: 16.88
instance: lr=3ce=5b=95h=5u=8 cost: 32.6
instance: lr=3ce=5b=495h=5u=4 cost: 16.88
instance: lr=3ce=5b=495h=5u=8 cost: 32.6
instance: lr=3ce=10b=95h=5u=4 cost: 20.34
instance: lr=3ce=10b=95h=5u=8 cost: 38.61
instance: lr=3ce=10b=495h=5u=4 cost: 20.34
instance: lr=3ce=10b=495h=5u=8 cost: 38.89
instance: lr=3ce=20b=95h=5u=4 cost: 24.3
instance: lr=3ce=20b=95h=5u=8 cost: 44.44
instance: lr=3ce=20b=495h=5u=4 cost: 24.34
instance: lr=3ce=20b=495h=5u=8 cos

In [21]:
%qtconsole

### Exact cost calculation CDI

In [24]:
for lr in lr_list:
    for ce in ce_list:
        for b in reversed(b_list):
            for d_max in d_max_list:
                instance = f'lr={lr}ce={ce}b={b}h={h}u={d_max}'
                f_name = f'dp_state_output_' + instance + '.p'
                try:
                    f_name = os.path.join('../', 'sourcing_models', 'dp_state_output', f_name)
                    qf_dp = pickle.load(open(f_name, 'rb'))
                    if ((0,)*lr) in qf_dp:
                        key = (0,)*lr
                    else:
                        key = int(len(qf_dp)/2)
                    visited = bfs(key)
                    qf_dp2 = {x:y for x, y in qf_dp.items() if x in visited}
                    
                    qf_cdi = {}
                    u1, u2, u3, cost = cdi_optimal[instance]
                    for state in qf_dp2:
                        qf_cdi[state] = get_cdi_action(state, u1, u2, u3)                
                    qf_cdi = uncover_cdi_states(qf_cdi)
                    cost = exact_cost(qf_cdi)
                    print(f'instance: {instance} cost: {round(cost, 2)}')
                except(OSError, IOError) as e:
                    print(f'file {f_name} does not exist.')

instance: lr=2ce=5b=95h=5u=4 cost: 16.85
instance: lr=2ce=5b=95h=5u=8 cost: 32.29
instance: lr=2ce=5b=495h=5u=4 cost: 16.85
instance: lr=2ce=5b=495h=5u=8 cost: 32.29
instance: lr=2ce=10b=95h=5u=4 cost: 19.8
instance: lr=2ce=10b=95h=5u=8 cost: 37.87
instance: lr=2ce=10b=495h=5u=4 cost: 19.8
instance: lr=2ce=10b=495h=5u=8 cost: 37.87
instance: lr=2ce=20b=95h=5u=4 cost: 23.35
instance: lr=2ce=20b=95h=5u=8 cost: 41.86
instance: lr=2ce=20b=495h=5u=4 cost: 23.25
instance: lr=2ce=20b=495h=5u=8 cost: 43.81
instance: lr=3ce=5b=95h=5u=4 cost: 16.88
instance: lr=3ce=5b=95h=5u=8 cost: 32.93
instance: lr=3ce=5b=495h=5u=4 cost: 16.88
instance: lr=3ce=5b=495h=5u=8 cost: 32.93
instance: lr=3ce=10b=95h=5u=4 cost: 20.47
instance: lr=3ce=10b=95h=5u=8 cost: 39.26
instance: lr=3ce=10b=495h=5u=4 cost: 20.47
instance: lr=3ce=10b=495h=5u=8 cost: 39.42
instance: lr=3ce=20b=95h=5u=4 cost: 25.0
instance: lr=3ce=20b=95h=5u=8 cost: 44.9
instance: lr=3ce=20b=495h=5u=4 cost: 24.4
instance: lr=3ce=20b=495h=5u=8 cost:

### Exact search CDI

In [30]:
def exact_search_cdi(instance, u1_arr, u2_arr, u3_arr, qf_dp2):
    min_cost = 1e9

    optimal_u1 = 0
    optimal_u2 = 0
    optimal_u3 = 0

    for u1 in u1_arr:
        for u2 in u2_arr:
            for u3 in u3_arr:
                qf_cdi = {}
                for state in qf_dp2:
                    qf_cdi[state] = get_cdi_action(state, u1, u2, u3)                
                qf_cdi = uncover_cdi_states(qf_cdi)
                cost_tmp = exact_cost(qf_cdi)
                if cost_tmp < min_cost:
                    optimal_u1 = u1
                    optimal_u2 = u2
                    optimal_u3 = u3
                    min_cost = cost_tmp
    return optimal_u1, optimal_u2, optimal_u3, min_cost

In [31]:
cdi_exact_optimal = {}
for lr in lr_list:
    for ce in ce_list:
        for b in reversed(b_list):
            for d_max in d_max_list:
                instance = f'lr={lr}ce={ce}b={b}h={h}u={d_max}'
                f_name = f'dp_state_output_' + instance + '.p'
                try:
                    f_name = os.path.join('../', 'sourcing_models', 'dp_state_output', f_name)
                    qf_dp = pickle.load(open(f_name, 'rb'))
                    if ((0,)*lr) in qf_dp:
                        key = (0,)*lr
                    else:
                        key = int(len(qf_dp)/2)
                    visited = bfs(key)
                    qf_dp2 = {x:y for x, y in qf_dp.items() if x in visited}
                    gamma, sigma = find_gamma_sigma()
                    u1_arr, u2_arr, u3_arr = find_cdi_param_arrays(gamma, sigma)
                    u1, u2, u3, cost = exact_search_cdi(instance, u1_arr, u2_arr, u3_arr, qf_dp2)
                    cdi_exact_optimal[instance] = (u1, u2, u3, cost)
                    print(f'Instance: {instance} Cost: {cost} Solution: {(u1, u2, u3)}')
                except(OSError, IOError) as e:
                    print(f'file {f_name} does not exist.')

Instance: lr=2ce=5b=95h=5u=4 Cost: 16.848404255319146 Solution: (4, 8, 1)
Instance: lr=2ce=5b=95h=5u=8 Cost: 32.28891688578486 Solution: (8, 15, 3)
Instance: lr=2ce=5b=495h=5u=4 Cost: 16.848404255319146 Solution: (4, 8, 1)
Instance: lr=2ce=5b=495h=5u=8 Cost: 32.28891688578486 Solution: (8, 15, 3)
Instance: lr=2ce=10b=95h=5u=4 Cost: 19.799405646359585 Solution: (4, 8, 2)
Instance: lr=2ce=10b=95h=5u=8 Cost: 37.41599855798702 Solution: (7, 17, 5)
Instance: lr=2ce=10b=495h=5u=4 Cost: 19.799405646359585 Solution: (4, 8, 2)
Instance: lr=2ce=10b=495h=5u=8 Cost: 37.865261706203384 Solution: (8, 17, 4)
Instance: lr=2ce=20b=95h=5u=4 Cost: 23.049999999999997 Solution: (3, 10, 3)
Instance: lr=2ce=20b=95h=5u=8 Cost: 41.76378600823044 Solution: (7, 18, 5)
Instance: lr=2ce=20b=495h=5u=4 Cost: 23.24556213017751 Solution: (4, 9, 3)
Instance: lr=2ce=20b=495h=5u=8 Cost: 43.80631001371743 Solution: (8, 19, 5)
Instance: lr=3ce=5b=95h=5u=4 Cost: 16.878306878306883 Solution: (4, 9, 1)
Instance: lr=3ce=5b=95h